In [1]:
import pandas as pd
import numpy as np
import nltk
# Cleaning the texts
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import RandomOverSampler
from joblib import dump, load
import pickle

Using TensorFlow backend.


In [2]:
leads = pd.read_excel('1000 leads.xlsx',encoding = 'ISO-8859-1')

In [3]:
leads.head()

,Lead Name,Location,Status,Status information,Unnamed: 4
0,Raja,hyderabad,Not Converted,"14/8/prema: share me details, available in evn...",NaN
1,Anirudh Reddy,pune,Not Converted,"14/8/prema: cal me tmrw, shared details to ema...",NaN
2,Sapna Dewani,bangalore,Converted,16|AuG|moHan:rnr,NaN
3,suresh,mumbai,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...,NaN
4,Akshay Shinde,hyderabad,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ...",NaN


In [4]:
leads.columns

Index(['Lead Name', 'Location', 'Status ', 'Status information', 'Unnamed: 4'], dtype='object')

In [5]:
leads = leads.drop(['Lead Name','Location','Unnamed: 4'],axis=1)

In [6]:
leads.head()

,Status,Status information
0,Not Converted,"14/8/prema: share me details, available in evn..."
1,Not Converted,"14/8/prema: cal me tmrw, shared details to ema..."
2,Converted,16|AuG|moHan:rnr
3,Not Converted,14/8/17(Surendra):i want only Server 16|AuG|mo...
4,Not Converted,"14/8/prema:rnr 16/8/prema: gave info, he said ..."


In [7]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 2 columns):
Status                998 non-null object
Status information    977 non-null object
dtypes: object(2)
memory usage: 15.8+ KB


In [8]:
leads['Status '].value_counts()

Not Converted    856
Converted        124
NOt Converted     11
Conveted           7
Name: Status , dtype: int64

In [9]:
leads['Status ']= leads['Status '].replace(['Conveted'],'Converted ')

In [10]:
leads['Status '].unique()

array(['Not Converted', 'Converted ', 'NOt Converted', nan], dtype=object)

In [11]:
leads = leads.dropna()

In [12]:
leads['status'] = leads['Status ']
leads['status_information'] = leads['Status information']

In [13]:
leads = leads.drop(['Status ','Status information'],axis=1)

In [14]:
leads['status'].unique()

array(['Not Converted', 'Converted ', 'NOt Converted'], dtype=object)

In [15]:
leads.reset_index(inplace = True) 

In [16]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
lem = WordNetLemmatizer()
corpus_lem = []
for i in range(0, len(leads)):
    review = re.sub('[^a-zA-Z]', ' ', leads['status_information'][i])
    review = review.lower()
    review = review.split()
    
    review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_lem.append(review)

In [17]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
lem = WordNetLemmatizer()
corpus_stem = []
for i in range(0, len(leads)):
    review = re.sub('[^a-zA-Z]', ' ', leads['status_information'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus_stem.append(review)

In [18]:
corpus_lem[0]

'prema share detail available evng prema postponed plan training currently prema interested'

In [19]:
corpus_stem[0]

'prema share detail avail evng prema postpon plan train current prema interest'

In [20]:
type(corpus_stem)

list

In [21]:
leads['status']=leads['status'].str.lower()

In [22]:
leads['status'].value_counts()

not converted    845
converted        129
Name: status, dtype: int64

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus_lem)

Y=leads['status']


In [24]:
pickle.dump(cv,open('transform_new.pkl','wb'))

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
tfidf=TfidfTransformer()

X=tfidf.fit_transform(X)

In [27]:
print(X.shape,Y.shape)

(974, 1105) (974,)


In [28]:
type(X)

scipy.sparse.csr.csr_matrix

In [29]:
pickle.dump(tfidf,open('transform.pkl','wb'))

In [30]:
ros = RandomOverSampler(random_state=0)
X_res,Y_res=ros.fit_resample(X, Y)

pickle.dump(ros,open('ros.pkl','wb'))

In [31]:
Y_res.value_counts(normalize=True) * 100

not converted    50.0
converted        50.0
Name: status, dtype: float64

In [32]:
Y_res=Y_res.replace(to_replace = ['converted ','not converted'], value = [0,1])

In [33]:
Y_res.value_counts(normalize=True) * 100

1    50.0
0    50.0
Name: status, dtype: float64

In [34]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(X_res,Y_res, test_size = 0.1, random_state = 1500)


In [35]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
y_pred=clf.predict(x_test)

In [37]:
pickle.dump(clf,open('clf.pkl','wb'))

In [38]:
y_pred

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1], dtype=int64)

In [39]:
expected = y_test

from sklearn import metrics
print(metrics.classification_report(expected, y_pred))
print(metrics.confusion_matrix(expected, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83        85
           1       0.84      0.80      0.82        84

    accuracy                           0.82       169
   macro avg       0.82      0.82      0.82       169
weighted avg       0.82      0.82      0.82       169

[[72 13]
 [17 67]]
